In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set();

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/katarina74/ml_lessons/main/lesson_2/data/techparams_train.csv")
X = df.drop(["target", "index"], axis=1)
y = df[["target"]]

In [3]:
poly_features = PolynomialFeatures(degree=2, include_bias = False)
X_poly = poly_features.fit_transform(X)

In [4]:
poly_train_X, poly_test_X, train_y, test_y = train_test_split(X_poly, y, test_size=0.33, random_state=42)

In [5]:
import shap
from sklearn.feature_selection import SelectFromModel

In [6]:
reg = LinearRegression().fit(poly_train_X, train_y)

In [7]:
shap_values = shap.LinearExplainer(reg, poly_train_X).shap_values(poly_train_X)
mean_shap = np.mean(np.abs(shap_values), axis=1)
thr = np.mean(mean_shap)
feature_selector = SelectFromModel(reg, threshold=thr, prefit=True)
selected_X = feature_selector.transform(poly_train_X)

In [8]:
select = LinearRegression().fit(selected_X, train_y)

In [9]:
selected_test_X = feature_selector.transform(poly_test_X)

In [10]:
pred = select.predict(selected_test_X)
pred

array([[2423.35576443],
       [2710.97452595],
       [2632.1665972 ],
       ...,
       [2507.3112144 ],
       [2577.73029322],
       [2483.66724875]])

In [11]:
score = select.score(selected_test_X, test_y)
mse = mean_squared_error(test_y, pred)
print(score)
print()
print(mse)

0.40041227284257286

34741.63834062006


In [12]:
#показатели модели без полиномиальных признаков и отбора
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
reg = LinearRegression().fit(train_X, train_y)

In [14]:
pred = reg.predict(test_X)
pred

array([[2357.32561418],
       [2671.44254732],
       [2594.490395  ],
       ...,
       [2496.45737597],
       [2558.81232783],
       [2490.53244122]])

In [15]:
score = reg.score(test_X, test_y)
mse = mean_squared_error(test_y, pred)
print(score)
print()
print(mse)

0.46471537322321455

31015.7531091195
